In [1]:
!pip install trl transformers accelerate peft datasets bitsandbytes einops -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 17.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 63.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 41.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import TextStreamer
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [3]:
# "heliosbrahma/mental_health_chatbot_dataset"
data_set_name = "Amod/mental_health_counseling_conversations"

In [4]:
data = load_dataset(data_set_name)
data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [5]:
print(data["train"][0])

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someho

In [6]:
# model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [8]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
chat = []

# Iterate through each row in the 'train' split of your dataset
for row in data['train']:
  single_chat = []
  # Append the 'Context' as a user's message
  single_chat.append({
      "role": "user",
      "content": row['Context']
  })
  # Append the 'Response' as an assistant's message
  single_chat.append({
      "role": "assistant",
      "content": row['Response']
  })

  # Append the single_chat list to the chat list
  chat.append(single_chat)  

In [10]:
chat[0]

[{'role': 'user',
  'content': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?"},
 {'role': 'assistant',
  'content': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't fe

### Add second dataset

In [ ]:
for d in data:
    single_chat = []
    user = data[2]['inputs'].split("###Instruction: ")[
        1].split("###Response: ")[0]
    assistant = data[2]['inputs'].split("###Instruction: ")[
        1].split("###Response: ")[1]

    single_chat.append({
        "role": "user",
        "content": user
    })
    # Append the 'Response' as an assistant's message
    single_chat.append({
        "role": "assistant",
        "content": assistant
    })
    chat.append(single_chat)

In [11]:
from datasets import Dataset
dataset = Dataset.from_dict({"chat": chat})
dataset = dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)})
print(dataset['formatted_chat'][0])

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

<s>[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone? [/INST]If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad f

In [12]:
dataset

Dataset({
    features: ['chat', 'formatted_chat'],
    num_rows: 3512
})

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

In [15]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, peft_config)

In [16]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=32, target_modules={'q_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [17]:
output_dir = "./output/mental-health-chat"
per_device_train_batch_size = 16 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 4  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 320        # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to = "none"
)

In [18]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="formatted_chat",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [20]:
peft_model.config.use_cache = False
trainer.train()

Step,Training Loss
10,3.004200
20,2.474000
30,2.235100
40,2.207900
50,2.131700
60,2.089300
70,2.064900
80,2.023100
90,2.012800
100,1.959800


TrainOutput(global_step=320, training_loss=1.8345550119876861, metrics={'train_runtime': 5191.9957, 'train_samples_per_second': 3.945, 'train_steps_per_second': 0.062, 'total_flos': 2.7938758662212813e+17, 'train_loss': 1.8345550119876861, 'epoch': 5.82})

In [21]:
# Loading original model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
# Loading PEFT model
PEFT_MODEL = f'{output_dir}/checkpoint-320'

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [52]:
from transformers import pipeline

# Function to generate or update the summary of the conversation history
def update_summary(conversation_history):
    # This is a placeholder function. You might use a summarization model here.
    # For example, using a pipeline for summarization (adjust model checkpoint as needed):
    summarizer = pipeline(
        "summarization", model="summarization-model-checkpoint")
    summary = summarizer(" ".join(conversation_history),
                         max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text']

# Modified get_query function to use summary as context
def get_query(summary, query):
    return (
        "<s>You are an assistant for mental health."
        "\nBelow is a summary of the conversation and a new instruction."
        "\nWrite a response that is concise and to the point. Remember, people prefer brief sentences."
        "\nPlease first identify the problem and state that, show empathy."
        "\nAlso finish with a question that you like to ask."
        "Summary of conversation: {}"
        "\n[INST]{}[/INST]\n</s>\n\n".format(summary, query)
    )


# Example usage
conversation_history = []

# Before making a new query, update the summary of the conversation
summary = update_summary(conversation_history)

# Then, use the updated summary as context for the new query
query = "The user reports feeling anxious."
eval_prompt = get_query(summary, query)

# Generate a response using the updated eval_prompt
def get_answer(context, query):
    eval_promt = get_query(context, query)
    encoding = tokenizer(eval_promt, return_tensors="pt")
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**encoding, streamer = text_streamer, max_new_tokens = 512)

In [53]:
get_answer("", "How can I prevent anxiety and depression?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health.
Below is an instruction that describes a patient's problem. please use previous conversation as context to answer
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to askPrevious conversation context: 
[INST]How can I prevent anxiety and depression?[/INST]
</s> 

I understand that you're looking for ways to prevent anxiety and depression. It's important to prioritize self-care and maintain good mental health. Here are some steps you can take:

1. Regular physical activity: Exercise regularly to help boost your mood and reduce stress.
2. Balanced diet: Eat a healthy diet to fuel your body and mind.
3. Adequate sleep: Ensure you're getting enough restful sleep each night.
4. Connect with others: Build strong relationships and maintain social connections.
5. Mindfulness practices: Incorporate mind

In [47]:
get_answer("I am feeling depressed about my career progress, i feel like I am not growing")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health. Below is an instruction that describes a patient's problem.
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to ask
[INST]I am feeling depressed about my career progress, i feel like I am not growing[/INST]
</s> 

I'm sorry to hear that you're feeling depressed about your career progress and that you feel like you're not growing. It's important to remember that everyone's career path is unique and that setbacks and challenges are a normal part of the journey. I'd be happy to help you explore ways to address these feelings and find new opportunities for growth.

Can you tell me more about what specifically is making you feel this way? What are some of the things you've tried in the past to improve your situation?</s>


In [48]:
get_answer("Hello I am Mira. I am recent mom, why i feel blue now a days? could you please help?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health. Below is an instruction that describes a patient's problem.
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to ask
[INST]Hello I am Mira. I am recent mom, why i feel blue now a days? could you please help?[/INST]
</s> 

Hello Mira, I'm here to help. I'm sorry to hear that you're feeling blue lately. It's common for new mothers to experience mood swings and feelings of sadness or anxiety after giving birth. This is known as the "baby blues." It's important to remember that you're not alone and that these feelings are normal. Would you like to talk about what you're going through or if there's anything specific that's been making you feel this way? Additionally, have you been getting enough rest, eating well, and taking care of yourself? It might be helpful to explore these areas as well. Is th

In [49]:
get_answer("Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health. Below is an instruction that describes a patient's problem.
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to ask
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s> 

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving your mood and overall mental health? For instance, regular exercise, a healthy diet, 

In [54]:
context = """
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s> 

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving your mood and overall mental health? For instance, regular exercise, a healthy diet, adequate sleep, and engaging in activities you enjoy can all contribute to better mental health. Do you have any specific interests or hobbies that you've been putting off due to your depression? I'd be happy to help you explore ways to incorporate these activities back into your life.

Additionally, have you considered seeking professional help from a therapist or counselor? They can provide you with evidence-based treatments and support to help you manage your depression. Would you be open to giving this a try?</s>
"""

In [55]:
get_answer(context, "I understand, but I feel better in coitus")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> You are an assistant for mental health.
Below is an instruction that describes a patient's problem. please use previous conversation as context to answer
Write a response that is concise and to the point. Remember, people prefer brief sentences.
Please first identify the problem and state that, show empathy.
Also finish with a question that you like to askPrevious conversation context: 
[INST]Hello I am Mira. I feel like coitus is helpful during my depression, how would you recommend to make it better?[/INST]
</s>  

Hi Mira, I'm here to help. I understand that you're experiencing depression and you find that coitus is helpful. It's important to note that while sexual activity can have positive effects on mood, it's not a cure for depression. If you're feeling depressed, it might be helpful to speak with a mental health professional. They can provide you with effective treatments and strategies to manage your depression. Would you be open to exploring other options for improving